In [5]:
import json
import networkx as nx
import numpy as np
import math
from numpy.linalg import norm

In [6]:
def showGraph(G):
    pos = {}
    for node in G.nodes(data = True):
        pos[node[0]] = [int(p) for p in node[1]['position'][0:2]]
    nx.draw(G, pos = pos)

def createGraph(graph):
    G = nx.Graph()
    for node in graph['node']:
        G.add_node(node['storeName'], 
            storeName = node['storeName'], 
            position = (node['posX'], node['posY']))
    # for edge in graph['edge']:
        # x1,y1,z1 = map(int,G.nodes[edge['node'][0]]['position'])
        # x2,y2,z2 = map(int,G.nodes[edge['node'][1]]['position'])
        # G.add_edge(edge['node'][0], edge['node'][1], weight = int(abs((x1-x2) +(y1-y2) +(z1-z2))))
    return G

def alignNodes(path):
    G = []
    f = open(path)
    graph = json.load(f, parse_int=str)
    for node in graph['storeNodes']:
        G.append([node['storeName'],int(node['posX']),int(node['posY'])])
    for i in range(len(G)):
        for j in range(i+1, len(G)):
            for k in range(1,3):
                if(1 <= abs(G[i][1]-G[j][1]) < 2):
                    G[i][k] = G[j][k] = (G[i][k]+G[j][k])/2
    for i in range(len(G)):
        for j in range(i+1, len(G)):
            for k in range(1,3):
                if(abs(G[i][k]-G[j][k]) < 2):
                    G[i][k] = G[j][k] = int((G[i][k]+G[j][k])//2)
                    # print(G[i],G[j]) 
    return G


In [7]:
alignedStoreNodes = alignNodes('./Jsons/S1.json')       
StoreNode = np.array([[int(X[1]),int(X[2])] for X in alignedStoreNodes ])      
for node in StoreNode:
    print(node)

[7 4]
[12  4]
[17  4]
[36  8]
[36 15]
[33 25]
[19 25]
[10 25]
[ 3 22]
[ 3 15]
[3 8]
[12 15]
[19 13]
[27 13]
[27 17]
[19 17]


In [8]:
for i in range(len(alignedStoreNodes)):
    for j in range(i+1,len(alignedStoreNodes)):
        if(not(alignedStoreNodes[i][1] == alignedStoreNodes[j][1] or alignedStoreNodes[i][2] == alignedStoreNodes[j][2] )):
            # print(alignedStoreNodes[i],alignedStoreNodes[j])
            pass

In [9]:
G = alignNodes('./Jsons/C1.json')
G = sorted(G,key=lambda x: x[1])
C = []
for node in G:
    print(node)
    # return G 
# for i in range(len(G)):
#     for j in range(i+1, len(G)):
#         if(not(G[i][1] == G[j][1] or G[i][2] == G[j][2])):
#             print(G[i],G[j])
            
for i in range(len(G)):
    for j in range(i+1, len(G)):
        if(not(G[i][1] == G[j][1] or G[i][2] == G[j][2])):
            if(not ([G[i][1],G[j][2]] in C)):
                C.append([G[i][1],G[j][2]])
            if(not([G[j][1],G[i][2]] in C)):
                C.append([G[j][1],G[i][2]])
center = [0,0]
for i in C:
    center[0] += i[0]
    center[1] += i[1]

center = [int(center[0]/5),int(center[1]/5)]
C.pop(C.index(center))
print(C)
C = np.array(C)
# print(type(C))

['C2', 7, 13]
['C1', 19, 7]
['C3', 19, 21]
['C4', 32, 13]
[[7, 7], [7, 21], [32, 7], [32, 21]]


In [13]:
D = []
for i in range(len(C)):
    for j in range(i+1,len(C)):
        if(C[i][0]==C[j][0] ):
            print("Y-axis",C[i],C[j],StoreNode[0])
            print(norm(np.cross(C[j]-C[i], C[i]-StoreNode[1]))/norm(C[j]-C[i]))
        if(C[i][1]==C[j][1] ):
            print("X-axis",C[i],C[j],StoreNode[0])
            print(norm(np.cross(C[j]-C[i], C[i]-StoreNode[1]))/norm(C[j]-C[i]))
# print(D)
# d = norm(np.cross(p2-p1, p1-p3))/norm(p2-p1)

Y-axis [7 7] [ 7 21] [7 4]
5.0
X-axis [7 7] [32  7] [7 4]
3.0
X-axis [ 7 21] [32 21] [7 4]
17.0
Y-axis [32  7] [32 21] [7 4]
20.0
